## RECOMMENDER - PHASE 1

**SUMMARY:**

We will be constructing a first run recommender to test our data based solely on correlation between ratings and wine.  This will help confirm that the dataframes we have assembled will be compatible when subjected to a recommender operation.  Since this will be a basic recommender, we will be limiting our input to only the key elements necessary for this exercise.

#### IMPORT PACKAGES

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

#### IMPORT WINE CATALOG AND USER FILES AND PREP FOR PIVOT TABLE

In [2]:
wine = pd.read_csv('./data/wine_catalog.csv')
wine = wine.fillna("")
wine.head()

,Unnamed: 0,winery,wine,rating,wine_style,wine_category,vintage,region,country,price,contact_for_price,wine_style_code,wine_category_code,region_code,country_code
0,0,Caymus,Cabernet Sauvignon N.V.,4.6,Cabernet Sauvignon,Red,N.V.,Napa Valley,United States,79.05,,15,2,587,36
1,1,The Prisoner,The Prisoner N.V.,4.4,Blend,Red,N.V.,Napa Valley,United States,39.75,,10,2,587,36
2,2,Masi,Costasera Amarone della Valpolicella Classico ...,4.3,Valpolicella,Red,N.V.,Amarone della Valpolicella Classico,Italy,54.99,,86,2,34,19
3,3,Stag's Leap Wine Cellars,ARTEMIS Cabernet Sauvignon N.V.,4.3,Cabernet Sauvignon,Red,N.V.,Napa Valley,United States,59.99,,15,2,587,36
4,4,Banfi,Brunello di Montalcino N.V.,4.2,Brunello,Red,N.V.,Brunello di Montalcino,Italy,65.98,,12,2,120,19


In [3]:
wine.drop(['Unnamed: 0','winery','wine_style','wine_category','vintage','region','country','price','contact_for_price','wine_style_code','wine_category_code','region_code','country_code'],axis=1,inplace=True)

In [4]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33163 entries, 0 to 33162
Data columns (total 2 columns):
wine      33163 non-null object
rating    33163 non-null float64
dtypes: float64(1), object(1)
memory usage: 518.3+ KB


In [5]:
wine=wine.rename(columns = {'wine':'wine_label'})
wine.head()

,wine_label,rating
0,Cabernet Sauvignon N.V.,4.6
1,The Prisoner N.V.,4.4
2,Costasera Amarone della Valpolicella Classico ...,4.3
3,ARTEMIS Cabernet Sauvignon N.V.,4.3
4,Brunello di Montalcino N.V.,4.2


In [6]:
#NOTE: While normally we would address NaN's in any other DataFrame, replacing them with zeros in a recommender adversely affects performance.

ratings = pd.read_csv('./data/user.csv')
#ratings = ratings.fillna(0.0)
ratings.head()

,Unnamed: 0,user-0,user-1,user-2,user-3,user-4,user-5,user-6,user-7,user-8,...,user-990,user-991,user-992,user-993,user-994,user-995,user-996,user-997,user-998,user-999
0,0,NaN,NaN,4.7,4.6,4.9,4.8,NaN,4.3,NaN,...,NaN,NaN,NaN,4.8,4.4,4.7,NaN,4.8,NaN,4.6
1,1,4.5,4.3,NaN,NaN,NaN,NaN,4.4,NaN,4.2,...,NaN,NaN,NaN,NaN,4.1,NaN,4.4,4.1,4.2,4.3
2,2,NaN,4.1,NaN,NaN,4.2,4.6,NaN,NaN,NaN,...,4.3,4.2,NaN,4.3,NaN,4.5,4.0,NaN,NaN,NaN
3,3,NaN,NaN,4.5,NaN,NaN,4.3,NaN,NaN,4.5,...,NaN,4.2,4.1,4.3,4.6,4.2,4.5,NaN,4.2,NaN
4,4,4.3,4.2,NaN,4.4,4.1,4.2,NaN,NaN,NaN,...,4.4,NaN,NaN,4.1,NaN,4.5,4.3,4.0,NaN,4.2


In [7]:
ratings.drop(['Unnamed: 0'],axis=1, inplace=True)

In [8]:
wine.insert(0,'wine_id',wine.index+1)
ratings.insert(0,'wine_id',ratings.index+1)

In [9]:
#going to use pd.melt() to to massage both DataFrames into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are unpivoted to the row axis, leaving just two non-identifier columns, variable and value.
wine_1 = pd.melt(wine,id_vars=['wine_label'],var_name="wine_style",value_name="wine_id")

In [10]:
wine_1.head(100)

,wine_label,wine_style,wine_id
0,Cabernet Sauvignon N.V.,wine_id,1.0
1,The Prisoner N.V.,wine_id,2.0
2,Costasera Amarone della Valpolicella Classico ...,wine_id,3.0
3,ARTEMIS Cabernet Sauvignon N.V.,wine_id,4.0
4,Brunello di Montalcino N.V.,wine_id,5.0
...,...,...,...
95,Gran Enemigo N.V.,wine_id,96.0
96,Ribera del Duero Tinto N.V.,wine_id,97.0
97,Pinotage N.V.,wine_id,98.0
98,Morgon N.V.,wine_id,99.0


In [11]:
ratings_1=pd.melt(ratings,id_vars=['wine_id'], var_name='user',value_name="rating")

In [12]:
ratings_1.head()

,wine_id,user,rating
0,1,user-0,NaN
1,2,user-0,4.5
2,3,user-0,NaN
3,4,user-0,NaN
4,5,user-0,4.3


#### CONSTRUCT THE PIVOT TABLE

In [13]:
reco_1 = pd.merge(wine_1, ratings_1, on='wine_id')
reco_1.shape

C:\Users\tonyb\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1089: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


(36944000, 5)

In [14]:
reco_1.head()

,wine_label,wine_style,wine_id,user,rating
0,Cabernet Sauvignon N.V.,wine_id,1.0,user-0,NaN
1,Cabernet Sauvignon N.V.,wine_id,1.0,user-1,NaN
2,Cabernet Sauvignon N.V.,wine_id,1.0,user-2,4.7
3,Cabernet Sauvignon N.V.,wine_id,1.0,user-3,4.6
4,Cabernet Sauvignon N.V.,wine_id,1.0,user-4,4.9


In [15]:
pivot = pd.pivot_table(reco_1, index='wine_label', columns='user', values='rating')
pivot.head()

user,user-0,user-1,user-10,user-100,user-101,user-102,user-103,user-104,user-105,user-106,...,user-990,user-991,user-992,user-993,user-994,user-995,user-996,user-997,user-998,user-999
wine_label,,,,,,,,,,,,,,,,,,,,,
#42 Meritage Red Blend 2016,3.30,NaN,3.800000,3.400000,NaN,3.60,NaN,3.500,3.70,NaN,...,3.600000,NaN,3.70,3.600,3.90,NaN,3.900000,3.6,NaN,NaN
'61 Franciacorta Nature 2010,3.85,4.300000,4.371429,4.033333,4.05,4.22,4.183333,4.100,4.25,4.085714,...,3.700000,4.083333,4.10,4.125,4.44,4.10,4.283333,4.1,4.150,3.700
'Giulio Ferrari' Riserva del Fondatore 1993,4.90,4.575000,4.633333,4.475000,4.30,4.60,4.300000,4.650,NaN,4.600000,...,4.700000,4.700000,4.40,4.450,4.45,4.90,4.533333,4.6,4.625,4.850
'Giulio Ferrari' Riserva del Fondatore 1997,4.90,NaN,4.550000,4.533333,4.45,NaN,4.900000,4.700,4.30,4.500000,...,4.633333,4.666667,4.75,4.650,4.60,4.85,4.650000,NaN,4.600,4.600
'Giulio Ferrari' Riserva del Fondatore 2004,4.50,4.466667,4.200000,4.600000,4.70,4.45,4.475000,4.625,4.50,4.400000,...,4.700000,4.650000,4.40,4.750,4.45,4.45,4.700000,4.6,4.400,4.375


In [16]:
pivot.shape

(10450, 1000)

#### CREATE THE SPARSE MATRIX

In [17]:
sparse_pivot=sparse.csr_matrix(pivot.fillna(0.0))

In [18]:
print(sparse_pivot)

  (0, 0)	3.3
  (0, 2)	3.8
  (0, 3)	3.4
  (0, 5)	3.6
  (0, 7)	3.5
  (0, 8)	3.7
  (0, 11)	3.7
  (0, 12)	3.5
  (0, 14)	3.3
  (0, 15)	3.9
  (0, 16)	3.6
  (0, 18)	3.4
  (0, 19)	3.5
  (0, 20)	3.7
  (0, 27)	3.4
  (0, 31)	3.5
  (0, 33)	3.3
  (0, 36)	3.3
  (0, 39)	3.4
  (0, 40)	3.4
  (0, 41)	3.5
  (0, 43)	3.4
  (0, 44)	3.3
  (0, 47)	3.6
  (0, 48)	3.4
  :	:
  (10449, 975)	3.95
  (10449, 976)	3.8
  (10449, 977)	3.7
  (10449, 978)	3.8499999999999996
  (10449, 979)	3.6500000000000004
  (10449, 980)	3.6500000000000004
  (10449, 981)	3.9333333333333336
  (10449, 982)	3.8
  (10449, 983)	4.066666666666666
  (10449, 984)	4.05
  (10449, 985)	4.1
  (10449, 986)	3.9
  (10449, 987)	3.8
  (10449, 988)	3.8499999999999996
  (10449, 989)	3.6500000000000004
  (10449, 990)	4.066666666666666
  (10449, 991)	3.8666666666666667
  (10449, 992)	4.2
  (10449, 993)	4.1
  (10449, 994)	3.9749999999999996
  (10449, 995)	3.75
  (10449, 996)	3.7
  (10449, 997)	4.2
  (10449, 998)	3.8333333333333335
  (10449, 999)	3.90000000000

#### CALCULATE COSINE SIMILARITY

Using Pairwise Distance, we will be set our metric to 'cosine' in order to be able to evaluate Cosine Similarity on a scale of 0 to 1, with our higher values deemed most similar.  If we were to set our metric to Euclidean within Pairwise Distance, then on a scale of 0 to 1 the lower values would be the most similar within the pivot table.  

In [19]:
recommender = pairwise_distances(sparse_pivot, metric='cosine')

#### BUILD THE RECOMMENDER

In [20]:
recommender_df = pd.DataFrame(recommender, columns=pivot.index, index=pivot.index)
recommender_df.head()

wine_label,#42 Meritage Red Blend 2016,'61 Franciacorta Nature 2010,'Giulio Ferrari' Riserva del Fondatore 1993,'Giulio Ferrari' Riserva del Fondatore 1997,'Giulio Ferrari' Riserva del Fondatore 2004,'Giulio Ferrari' Riserva del Fondatore 2005,'Giulio Ferrari' Riserva del Fondatore 2006,'Giulio Ferrari' Riserva del Fondatore 2007,'Rosso & Bianco' Bianco (Pinot Grigio) 2017,(B) Old Vine Zinfandel Lodi 2016,...,Ürziger Würzgarten Riesling Spätlese 2018,Ürziger Würzgarten Riesling Spätlese N.V.,ΓΙΟΡΤΗ (Feast) White 2018,Ξυνιστέρι (Xynisteri) 2018,ΩΔΗ ΠΑΝΟΣ (Ode Panos) N.V.,‘61 Franciacorta Brut 1961,‘61 Franciacorta Brut N.V.,‘61 Franciacorta Rosé 1961,‘61 Franciacorta Rosé N.V.,‘61 Franciacorta Satèn 1961
wine_label,,,,,,,,,,,,,,,,,,,,,
#42 Meritage Red Blend 2016,0.000000,0.307243,0.320212,0.313042,0.319982,0.311664,0.315161,0.316054,0.473012,0.487344,...,0.489250,0.346001,0.500101,0.493689,0.314372,0.314413,0.322874,0.333469,0.311157,0.311883
'61 Franciacorta Nature 2010,0.307243,0.000000,0.048162,0.047457,0.049482,0.046418,0.040357,0.046592,0.282304,0.295366,...,0.319757,0.079676,0.296630,0.310398,0.051346,0.049843,0.051451,0.058169,0.044640,0.045102
'Giulio Ferrari' Riserva del Fondatore 1993,0.320212,0.048162,0.000000,0.063390,0.062203,0.060993,0.062067,0.063703,0.289916,0.314535,...,0.328691,0.090053,0.309575,0.324033,0.067168,0.066136,0.068959,0.076833,0.059594,0.060693
'Giulio Ferrari' Riserva del Fondatore 1997,0.313042,0.047457,0.063390,0.000000,0.062010,0.060340,0.056132,0.058057,0.296826,0.310949,...,0.318012,0.091314,0.306676,0.320409,0.060469,0.061023,0.064917,0.070314,0.057315,0.060831
'Giulio Ferrari' Riserva del Fondatore 2004,0.319982,0.049482,0.062203,0.062010,0.000000,0.063007,0.062385,0.061310,0.291377,0.309192,...,0.327039,0.093383,0.314748,0.317847,0.065696,0.065423,0.063302,0.075994,0.059793,0.062109


In [21]:
recommender_df.describe()

wine_label,#42 Meritage Red Blend 2016,'61 Franciacorta Nature 2010,'Giulio Ferrari' Riserva del Fondatore 1993,'Giulio Ferrari' Riserva del Fondatore 1997,'Giulio Ferrari' Riserva del Fondatore 2004,'Giulio Ferrari' Riserva del Fondatore 2005,'Giulio Ferrari' Riserva del Fondatore 2006,'Giulio Ferrari' Riserva del Fondatore 2007,'Rosso & Bianco' Bianco (Pinot Grigio) 2017,(B) Old Vine Zinfandel Lodi 2016,...,Ürziger Würzgarten Riesling Spätlese 2018,Ürziger Würzgarten Riesling Spätlese N.V.,ΓΙΟΡΤΗ (Feast) White 2018,Ξυνιστέρι (Xynisteri) 2018,ΩΔΗ ΠΑΝΟΣ (Ode Panos) N.V.,‘61 Franciacorta Brut 1961,‘61 Franciacorta Brut N.V.,‘61 Franciacorta Rosé 1961,‘61 Franciacorta Rosé N.V.,‘61 Franciacorta Satèn 1961
count,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,...,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000,10450.000000
mean,0.416977,0.185607,0.200717,0.198100,0.200023,0.198835,0.196115,0.199489,0.395352,0.411780,...,0.427329,0.226078,0.409596,0.421169,0.202084,0.199922,0.202865,0.210877,0.196323,0.198089
std,0.080437,0.111871,0.109167,0.109625,0.109408,0.109557,0.109883,0.109270,0.083459,0.081365,...,0.079321,0.105869,0.081533,0.080127,0.109085,0.109307,0.108998,0.107660,0.109699,0.109487
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.339486,0.074160,0.094817,0.091931,0.093551,0.092375,0.089914,0.093966,0.314369,0.331453,...,0.350069,0.123159,0.329400,0.343018,0.096431,0.093979,0.096484,0.107811,0.090469,0.092450
50%,0.399098,0.152810,0.169463,0.166435,0.168205,0.167019,0.164506,0.168533,0.375670,0.392715,...,0.409201,0.196246,0.390772,0.402424,0.170818,0.168375,0.171045,0.181105,0.165033,0.166908
75%,0.498186,0.302809,0.314461,0.312456,0.314360,0.313427,0.310695,0.313283,0.480018,0.494182,...,0.507159,0.336198,0.492295,0.501862,0.315882,0.314276,0.316535,0.322857,0.310725,0.312288
max,0.565436,0.350493,0.362294,0.356145,0.356863,0.355642,0.352192,0.357670,0.539767,0.565400,...,0.568845,0.385345,0.556494,0.568944,0.362424,0.364516,0.362363,0.370638,0.358624,0.361599


In [22]:
## TEST THE RECOMMENDER

In [23]:
search = 'Comtes de Champagne Blanc de Blancs 1983'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 4.9025222816399285
Number of ratings 935

10 closest wine labels

wine_label
Gavi 2017                              0.033330
Lighea N.V.                            0.033335
La Combe Pilate 2016                   0.033372
Rutherford Cabernet Sauvignon 2014     0.033375
Cabernet Sauvignon (Signature) 2016    0.033390
Crémant de Loire Brut N.V.             0.033396
Old Vines White 2018                   0.033396
O.R. 1735 Brut Champagne 2004          0.033413
Dijon Clones Chardonnay 2014           0.033418
Village Chardonnay 2017                0.033419
Name: Comtes de Champagne Blanc de Blancs 1983, dtype: float64

*******************************************************************************************

 
Average rating 4.9025222816399285
Number of ratings 935

10 closest wine labels

wine_label
Gavi 2017                              0.033330
Lighea N.V.                            0.033335
La Combe Pilate 2016                   0.033372
Rutherford Cabernet Sauvignon 2

In [24]:
search = 'Reserva Tempranillo 2014'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 3.7071428571428635
Number of ratings 532

10 closest wine labels

wine_label
Brut Rosé Champagne 2004                0.270355
Franciacorta Rosé N.V.                  0.270846
Sauvignon Blanc 2015                    0.270858
Langhe Rosso N.V.                       0.270869
Extra Brut N.V.                         0.270870
Margaux (Grand Cru Classé) 2009         0.271042
Saint-Julien (Grand Cru Classé) 2009    0.271081
Bella Vigna Chardonnay N.V.             0.271108
Saint-Julien (Grand Cru Classé) 2006    0.271120
Bianco di Ampeleia 2018                 0.271128
Name: Reserva Tempranillo 2014, dtype: float64

*******************************************************************************************



In [25]:
search = 'Cabernet Sauvignon RBS Beckstoffer To Kalon'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 4.819496855345906
Number of ratings 477

10 closest wine labels

wine_label
Shiraz 2018                                  0.308251
Reserve Cabernet Sauvignon N.V.              0.308425
Châteauneuf-du-Pape Blanc N.V.               0.308543
Vintage Port 1963                            0.308611
Red N.V.                                     0.308675
Saint-Estèphe (Grand Cru Classé) 2005        0.308689
Sparkling Brut N.V.                          0.308753
Sauternes (Premier Grand Cru Classé) 2010    0.308766
Prosecco Treviso Brut N.V.                   0.308796
Corton-Charlemagne Grand Cru 2015            0.308817
Name: Cabernet Sauvignon RBS Beckstoffer To Kalon Vineyard 2016, dtype: float64

*******************************************************************************************



In [26]:
search = 'Naissance Sauvignon Blanc'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 4.093283582089557
Number of ratings 536

10 closest wine labels

wine_label
Amarone della Valpolicella N.V.          0.267119
Saint-Julien (Grand Cru Classé) 2015     0.267260
Reserva Tinto 2016                       0.267343
Cuvée Brut Champagne N.V.                0.267468
Saint-Estèphe (Grand Cru Classé) 2005    0.267523
Soave Classico N.V.                      0.267568
Brut Premier Champagne N.V.              0.267582
Vintage Port 1977                        0.267590
Blanc de Noirs N.V.                      0.267845
Brut Champagne 1998                      0.267860
Name: Naissance Sauvignon Blanc 2017, dtype: float64

*******************************************************************************************

 
Average rating 4.125426852929765
Number of ratings 859

10 closest wine labels

wine_label
Rosé de Saignée Brut Champagne N.V.               0.069350
Viognier 2017                                     0.069365
Riesling 2016                                    

In [27]:
search = 'Malbec'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 4.403355704697978
Number of ratings 745

10 closest wine labels

wine_label
Pomerol 2011                                 0.137030
Petite Sirah 2016                            0.137034
Sauternes (Premier Grand Cru Classé) 2003    0.137146
Chianti Classico 2015                        0.137151
Colheita Branco 2018                         0.137167
Gewürztraminer 2017                          0.137173
Barolo 2014                                  0.137192
Cabernet Sauvignon 2012                      0.137204
Bianco di Ampeleia 2018                      0.137217
Moscato d'Asti 2015                          0.137237
Name: Bramare Malbec Lujan de Cuyo N.V., dtype: float64

*******************************************************************************************

 
Average rating 3.7342235067710035
Number of ratings 1000

10 closest wine labels

wine_label
Cava Celler Batlle Gran Reserva Brut 2005                              0.000363
Moscato d'Asti N.V.                       

*******************************************************************************************

 
Average rating 3.794291609353505
Number of ratings 727

10 closest wine labels

wine_label
Amarone della Valpolicella N.V.                                           0.146632
Sparkling Shiraz N.V.                                                     0.147652
Haut-Médoc 2010                                                           0.147849
Brut Rosé 2012                                                            0.147927
Brut Champagne 2000                                                       0.147952
Sin Zin Zinfandel N.V.                                                    0.147961
Brunello di Montalcino N.V.                                               0.147969
Casal di Serra Verdicchio dei Castelli di Jesi Classico Superiore 2016    0.147986
Côtes du Rhône N.V.                                                       0.147986
Barrel Select Malbec N.V.                                          

Number of ratings 761

10 closest wine labels

wine_label
Châteauneuf-du-Pape 2015                0.128317
Vintage Port 1983                       0.128352
Vintage Port 2007                       0.128358
Pomerol 2011                            0.128394
Saint-Julien (Grand Cru Classé) 2010    0.128399
Crémant de Loire N.V.                   0.128427
Saint-Julien (Grand Cru Classé) 2000    0.128431
Pinot Gris 2018                         0.128433
Red N.V.                                0.128448
Sancerre La Bourgeoise Blanc 2016       0.128458
Name: Tinto (Malbec) 2010, dtype: float64

*******************************************************************************************

 
Average rating 3.7087044534413023
Number of ratings 741

10 closest wine labels

wine_label
Vintage Port 2009                                  0.138823
Blanc de Blancs Brut Champagne N.V.                0.139653
Saint-Émilion Grand Cru (Grand Cru Classé) 2015    0.139725
Saint-Julien (Grand Cru Classé) 2008      

Number of ratings 864

10 closest wine labels

wine_label
Viognier 2017                                     0.066400
La Montesa 2016                                   0.066811
Riesling 2016                                     0.067091
Rosé de Saignée Brut Champagne N.V.               0.067094
Blanc de Noirs Brut Champagne Premier Cru N.V.    0.067100
Alvarinho 2017                                    0.067148
Old Vine Zinfandel 2016                           0.067187
Albariño 2015                                     0.067189
Meursault Blanc 2014                              0.067225
Bendigo Estate Pinot Noir N.V.                    0.067328
Name: Malbec (I) 2017, dtype: float64

*******************************************************************************************

 
Average rating 3.8914518317503286
Number of ratings 737

10 closest wine labels

wine_label
Margaux (Grand Cru Classé) 2015         0.141602
Cava Brut Rosé N.V.                     0.141609
Meursault 2016              

Name: Father Malbec - Cabernet Franc N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.099934640522888
Number of ratings 765

10 closest wine labels

wine_label
Riesling Dry Red Slate 2015               0.125020
Sparkling Shiraz N.V.                     0.125216
Moscato Rosé N.V.                         0.125342
Crémant de Bourgogne Brut Réserve N.V.    0.125604
Vintage Port 1994                         0.125684
Brut Champagne 1998                       0.125770
Grüner Veltliner 2018                     0.125804
Pinot Gris N.V.                           0.125851
Saint-Julien (Grand Cru Classé) 2008      0.125890
Margaux (Grand Cru Classé) 2010           0.125904
Name: Single Vineyard (Las Compuertas) Malbec 2015, dtype: float64

*******************************************************************************************

 
Average rating 4.507840909090905
Number of ratings 880

10 closest wine labels

wi

...

Name: Juan Benegas Malbec 2017, dtype: float64

*******************************************************************************************

 
Average rating 3.655476403457648
Number of ratings 1000

10 closest wine labels

wine_label
Champ Cain Brut Champagne Grand Cru 'Avize' 2004                       0.000197
Blanc de Blancs Brut 2008                                              0.000199
Nuit Blanche Champagne N.V.                                            0.000199
Cuvée Joséphine Brut Champagne 2008                                    0.000201
Special Club Brut Champagne Premier Cru 2008                           0.000201
Extra Brut Champagne 2004                                              0.000202
Minéral Blanc de Blancs Extra Brut Champagne Grand Cru 'Avize' 2008    0.000203
Dream Vintage Brut Champagne Grand Cru 'Bouzy' 2004                    0.000204
Rosa di Rosa N.V.                                                      0.000204
Cava Celler Batlle Gran Reserva Brut 2005    

Average rating 3.897113402061853
Number of ratings 485

10 closest wine labels

wine_label
Saint-Julien (Grand Cru Classé) 1995                0.302913
Crémant de Limoux Brut N.V.                         0.302927
Pessac-Léognan (Grand Cru Classé de Graves) 2009    0.303635
Malbec 2016                                         0.303656
Pinot Noir 2018                                     0.303701
Sauternes (Premier Grand Cru Classé) 2007           0.303714
Brut Champagne 2004                                 0.303732
Vieris Sauvignon Blanc 2016                         0.303782
Sin Zin Zinfandel N.V.                              0.303877
Brut Nature Champagne N.V.                          0.303891
Name: Q Malbec 2017, dtype: float64

*******************************************************************************************

 
Average rating 3.6079681274900386
Number of ratings 502

10 closest wine labels

wine_label
Vintage Port 2009                                   0.291300
Chianti Classi

Name: Estate Malbec 2016, dtype: float64

*******************************************************************************************

 
Average rating 3.798848368522071
Number of ratings 521

10 closest wine labels

wine_label
Sauternes (Premier Grand Cru Classé) 2011    0.277383
Haut-Médoc (Grand Cru Classé) 2010           0.277878
Sonoma Coast Chardonnay 2016                 0.278039
Cuvée Brut N.V.                              0.278236
Village Chardonnay 2017                      0.278269
Pauillac (Grand Cru Classé) 2008             0.278346
Proprietary Red N.V.                         0.278361
Langhe Rosso N.V.                            0.278371
Merlot 2016                                  0.278463
Pinot Noir 2016                              0.278468
Name: Malbec - Tannat 2010, dtype: float64

*******************************************************************************************

 
Average rating 3.7342235067710035
Number of ratings 1000

10 closest wine labels

wine_label


*******************************************************************************************

 
Average rating 3.598694607305709
Number of ratings 999

10 closest wine labels

wine_label
Cuvée Joséphine Brut Champagne 2008                                     0.000830
Cabernet Sauvignon (Signature) 2016                                     0.000834
Special Club Brut Champagne Premier Cru 2008                            0.000834
Moscato d'Asti N.V.                                                     0.000836
Terroirs Blanc de Blancs Extra Brut Champagne Grand Cru 'Avize' 2010    0.000839
Brut Rosé 2009                                                          0.000839
Champ Cain Brut Champagne Grand Cru 'Avize' 2004                        0.000839
Nuit Blanche Champagne N.V.                                             0.000839
Blanc de Blancs Champagne Grand Cru N.V.                                0.000842
Dream Vintage Brut Champagne Grand Cru 'Bouzy' 2004                     0.000844
Name

Name: Tahuan Malbec 2017, dtype: float64

*******************************************************************************************

 
Average rating 3.9238866396761116
Number of ratings 494

10 closest wine labels

wine_label
Amarone della Valpolicella Classico N.V.                       0.295478
Rioja Gran Reserva 2011                                        0.296157
Rioja Gran Reserva 2010                                        0.296465
Vintage Port 1983                                              0.296608
Pomerol 2011                                                   0.296689
Blanc de Blancs 2011                                           0.296745
Les Genettes Pinot Noir Champagne Grand Cru 'Ambonnay' 2010    0.296759
Pauillac (Grand Cru Classé) 2014                               0.296810
Pinot Noir Brut Rosé N.V.                                      0.296849
Brunello di Montalcino 2006                                    0.296909
Name: Malbec Rosé 2018, dtype: float64

***********

In [28]:
search = 'Rioja Gran Reserva'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 4.11860509312049
Number of ratings 877

10 closest wine labels

wine_label
Viognier 2017                                     0.059751
Jumilla Silver Label N.V.                         0.059963
Blanc de Noirs Brut Champagne Premier Cru N.V.    0.060329
Crémant de Loire N.V.                             0.060448
Albariño 2015                                     0.060468
Sexy Beast Cabernet Sauvignon 2016                0.060635
Riesling 2016                                     0.060642
Icon Sauvignon Blanc 2017                         0.060651
Brut Nature Champagne N.V.                        0.060656
Baccanera 2015                                    0.060672
Name: Rioja Gran Reserva N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.101902100270997
Number of ratings 984

10 closest wine labels

wine_label
Sin Zin Zinfandel N.V.                                                  0.008484
Nuit

Number of ratings 968

10 closest wine labels

wine_label
Adèo Bolgheri 2015                      0.015789
Riva Ranch Chardonnay 2018              0.015802
Olivet Lane Vineyard Chardonnay 2016    0.015802
Riesling 2016                           0.015804
Albariño 2015                           0.015809
EQ Coastal Sauvignon Blanc 2018         0.015822
Sancerre Le Roc 2015                    0.015838
Vermentino Costa Toscana 2016           0.015857
Alvarinho 2018                          0.015861
Tiara White (San Alberto) 2017          0.015864
Name: Rioja Gran Reserva 2009, dtype: float64

*******************************************************************************************

 
Average rating 4.15903753443526
Number of ratings 968

10 closest wine labels

wine_label
Adèo Bolgheri 2015                      0.015789
Riva Ranch Chardonnay 2018              0.015802
Olivet Lane Vineyard Chardonnay 2016    0.015802
Riesling 2016                           0.015804
Albariño 2015           


 
Average rating 4.101902100270997
Number of ratings 984

10 closest wine labels

wine_label
Sin Zin Zinfandel N.V.                                                  0.008484
Nuit Blanche Champagne N.V.                                             0.008651
Les Deux Albion 'Principaute d'Orange' Vaucluse Blanc 2016              0.008652
Champ Cain Brut Champagne Grand Cru 'Avize' 2004                        0.008656
Blanco 2014                                                             0.008657
Family Crest Chardonnay 2016                                            0.008658
Cuvée Joséphine Brut Champagne 2008                                     0.008660
Dragon's Tooth 2016                                                     0.008662
Puligny-Montrachet 2017                                                 0.008663
Terroirs Blanc de Blancs Extra Brut Champagne Grand Cru 'Avize' 2010    0.008663
Name: Rioja Gran Reserva 2011, dtype: float64

*************************************************

In [36]:
search = 'Moscato'

for wine_label in wine.loc[wine['wine_label'].str.contains(search), 'wine_label']:
    print(' ')
    print('Average rating', pivot.loc[wine_label, :].mean())
    print('Number of ratings', pivot.T[wine_label].count())
    print('')
    print('10 closest wine labels')
    print('')
    print(recommender_df[wine_label].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

 
Average rating 4.401933183183184
Number of ratings 888

10 closest wine labels

wine_label
Brut Rosé 2015                                             0.057520
Saint-Émilion Grand Cru (Premier Grand Cru Classé) 2006    0.057735
Merlot N.V.                                                0.057991
Meursault 2016                                             0.058020
Meursault 2017                                             0.058054
Gavi 2017                                                  0.058091
Montunal 2018                                              0.058100
Il Ducale Pinot Grigio 2017                                0.058127
Brunello di Montalcino 2013                                0.058135
Brunello di Montalcino N.V.                                0.058144
Name: Moscato d'Oro 2017, dtype: float64

*******************************************************************************************

 
Average rating 4.239905533012737
Number of ratings 1000

10 closest wine labels

wine_labe

Name: Moscato N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.239905533012737
Number of ratings 1000

10 closest wine labels

wine_label
Brut Rosé Champagne 2012                               0.000118
Special Club Brut Champagne Premier Cru 2008           0.000119
Dream Vintage Brut Champagne Grand Cru 'Bouzy' 2004    0.000120
Blanc de Blancs Champagne Grand Cru N.V.               0.000122
Cava Celler Batlle Gran Reserva Brut 2005              0.000122
Vértice Pinot Noir N.V.                                0.000122
O.R. 1735 Brut Champagne 2004                          0.000123
Moscato d'Asti N.V.                                    0.000124
Rosa di Rosa N.V.                                      0.000124
Cuvée William Deutz Millesimé Brut Champagne 2008      0.000124
Name: Moscato N.V., dtype: float64

*******************************************************************************************

 
Averag

Number of ratings 516

10 closest wine labels

wine_label
Dizy-Terres Rouges Rosé Extra Brut Champagne 2009    0.280042
Albariño 2018                                        0.280595
Sparkling N.V.                                       0.280640
Crémant du Jura Brut N.V.                            0.281065
Sauternes (Premier Grand Cru Classé) 2011            0.281313
Red Blend 2014                                       0.281317
Extra Brut Rosé Champagne Premier Cru N.V.           0.281390
Saint-Estèphe (Grand Cru Classé) 2005                0.281477
The Wolftrap White Blend 2017                        0.281561
Sparkling Shiraz N.V.                                0.281577
Name: Moscato Rosa Schweizer 2013, dtype: float64

*******************************************************************************************

 
Average rating 4.199185336048885
Number of ratings 491

10 closest wine labels

wine_label
Extra Brut Rosé Champagne Premier Cru N.V.    0.298514
Sauternes (Premier Grand Cru C

Name: Moscato d'Asti N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.199583333333331
Number of ratings 940

10 closest wine labels

wine_label
Blanc de Blancs Brut Champagne N.V.    0.030933
Chardonnay 2014                        0.030945
Pomerol 2012                           0.030973
Primitivo (a.k.a. Zinfandel) N.V.      0.030975
Family Crest Chardonnay 2016           0.030978
Gavi 2017                              0.030978
Napa Valley Cabernet Sauvignon N.V.    0.031003
Crémant de Bourgogne Brut N.V.         0.031012
Brut Rosé Champagne N.V.               0.031015
Côtes du Rhône N.V.                    0.031016
Name: Cascinetta Moscato d'Asti N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.246580411100849
Number of ratings 1000

10 closest wine labels

wine_label
Brut Rosé Champagne 2012                          

Number of ratings 920

10 closest wine labels

wine_label
Meursault 2016                         0.041056
Brunello di Montalcino 2014            0.041073
Chablis 2018                           0.041164
Pinot Grigio 2018                      0.041188
Brut Champagne 2006                    0.041200
Cabernet Sauvignon 2017                0.041203
Napa Valley Cabernet Sauvignon N.V.    0.041206
Châteauneuf-du-Pape N.V.               0.041214
Merlot 2017                            0.041244
Chardonnay 2017                        0.041273
Name: Moscato d'Asti Nivole 2012, dtype: float64

*******************************************************************************************

 
Average rating 4.131589008357047
Number of ratings 1000

10 closest wine labels

wine_label
Sauvignon Blanc N.V.       0.000092
Sauvignon Blanc 2017       0.000109
Prosecco N.V.              0.000109
Chardonnay N.V.            0.000112
Chardonnay 2017            0.000113
Brut Rosé N.V.             0.000116
Sauvignon

...

Name: Moscato d'Asti 2015, dtype: float64

*******************************************************************************************

 
Average rating 4.195677027509823
Number of ratings 933

10 closest wine labels

wine_label
Sauternes (Premier Grand Cru Classé) 2003         0.034591
Brunello di Montalcino 2014                       0.034600
Pouilly-Fuissé Vieilles Vignes 'Alliance' 2017    0.034607
Brunello di Montalcino 2007                       0.034628
Barrel Select Malbec N.V.                         0.034642
La Valletta Piemonte Chardonnay 2010              0.034656
Sauvignon Blanc (Reserva) 2017                    0.034661
Zinfandel 2014                                    0.034664
Getariako Txakolina 2018                          0.034664
Cava Celler Batlle Gran Reserva Brut 2005         0.034665
Name: Passion Fruit - Moscato N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.199421296296294
N

 
Average rating 4.094023479188905
Number of ratings 937

10 closest wine labels

wine_label
Rioja Gran Reserva 2010        0.032407
Tahuan Malbec 2017             0.032454
Vintage Port 2003              0.032467
Bourgogne Chardonnay 2017      0.032481
Moscato d'Asti 2018            0.032487
Châteauneuf-du-Pape N.V.       0.032495
Vintage Port 2011              0.032539
Les Voisins Chardonnay 2016    0.032540
Gavi 2017                      0.032547
Blanco 2014                    0.032554
Name: Bricco Quaglia Moscato d'Asti N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.094137442437123
Number of ratings 941

10 closest wine labels

wine_label
Central Coast Pinot Noir N.V.                                 0.030177
Valpolicella N.V.                                             0.030381
Cabernet Sauvignon 2014                                       0.030382
Entre-deux-Mers N.V.                              


*******************************************************************************************

 
Average rating 4.032909934765364
Number of ratings 1000

10 closest wine labels

wine_label
Sauvignon Blanc N.V.       0.000094
Prosecco N.V.              0.000105
Chardonnay N.V.            0.000106
Brut Rosé N.V.             0.000112
Chardonnay 2017            0.000113
Sauvignon Blanc 2017       0.000118
Cabernet Sauvignon N.V.    0.000128
Pinot Grigio 2018          0.000135
Brut N.V.                  0.000143
Brut Champagne N.V.        0.000149
Name: Moscato d'Asti 2017, dtype: float64

*******************************************************************************************

 
Average rating 4.032909934765364
Number of ratings 1000

10 closest wine labels

wine_label
Sauvignon Blanc N.V.       0.000094
Prosecco N.V.              0.000105
Chardonnay N.V.            0.000106
Brut Rosé N.V.             0.000112
Chardonnay 2017            0.000113
Sauvignon Blanc 2017       0.000118
Cabern


wine_label
Barrel Select Malbec N.V.                                     0.029799
Ardèche Chardonnay 2017                                       0.029805
Riesling Alsace Réserve 2016                                  0.029867
Barbaresco N.V.                                               0.029912
Letter Series 'B' Sauvignon Blanc 2018                        0.029915
Brut Réserve Champagne N.V.                                   0.029922
Pinot Grigio 2018                                             0.029923
Bianco di Ampeleia 2018                                       0.029945
Cabernet Sauvignon 2016                                       0.029946
Les Deux Albion 'Principaute d'Orange' Vaucluse Blanc 2016    0.029949
Name: Dolcevita Moscato d'Asti N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.246462829736209
Number of ratings 973

10 closest wine labels

wine_label
Terroirs Blanc de Blancs Extra Brut Cha

Name: Moscato d'Asti N.V., dtype: float64

*******************************************************************************************

 
Average rating 4.246580411100849
Number of ratings 1000

10 closest wine labels

wine_label
Brut Rosé Champagne 2012                                                0.000038
Dream Vintage Brut Champagne Grand Cru 'Bouzy' 2004                     0.000039
Champ Cain Brut Champagne Grand Cru 'Avize' 2004                        0.000040
Spécial Club Brut Champagne 2008                                        0.000040
Cuvée Joséphine Brut Champagne 2008                                     0.000040
Brut Rosé 2009                                                          0.000040
Rosa di Rosa N.V.                                                       0.000040
Terroirs Blanc de Blancs Extra Brut Champagne Grand Cru 'Avize' 2010    0.000040
Extra Brut Champagne 2004                                               0.000041
Special Club Brut Champagne Premier Cru 2